<img align="left" src = logos.png width=400 style="padding: 10px"> 
<br>
<b>Getting lightcurves and cutouts of DIA sources: Supernovae</b> <br>
Contact author: Vincenzo Petrecca <br>
Last verified to run: 2022-11-18<br>
LSST Science Piplines version: Weekly 2022_40 <br>
Container Size: medium <br>

**Description:** This notebook shows how to extract light curves of Supernovae detected on difference images querying both the *truth* and the DIA tables. Then, it shows how to plot the `calexp`, the `template` and the `diffexp` images for each detection. Finally, it provides hints on how the execute the DIA on the RSP with user-defined templates.

**Credit**: Developed by Vincenzo Petrecca. Please consider acknowledging Vincenzo if this notebook is used for the preparation of journal articles, software releases, or other notebooks. This notebook is based in part on material originally developed by Leanne Guy, Melissa Graham, Jeff Carlin and the Rubin Community Engagement Team for Data Preview 0. 

Special thanks to Neven Caplar for updating the notebook by using the new dp0.2 truth table

## 0. Set Up

In [ ]:
### Rubin-specific packages

import lsst.geom as geom
import lsst.sphgeom as sphgeom
import lsst.daf.base as dafBase
import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure
import lsst.daf.butler as dafButler
from lsst.daf.butler import Butler
from rubin_jupyter_utils.lab.notebook import get_tap_service
service = get_tap_service()

### General python / astronomy packages

import matplotlib.pyplot as plt
import numpy as np
import math
import gc
from matplotlib import container
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.wcs import WCS
import astropy.units as u
from astropy.time import Time
from astropy.table import Table
from astropy.io import fits
from astropy.timeseries import LombScargle
from astropy.io import ascii
from scipy.interpolate import interp1d, splrep, splev

import time

In [ ]:
plt.style.use('tableau-colorblind10')
%matplotlib inline

In [ ]:
import warnings

# Prevent some helpful but ancillary warning messages from printing
#   during some LSST DM Release calls
warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # Get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # Clear the figure
    plt.close(fig)  # Close the figure
    gc.collect()    # Call the garbage collector

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 13,
          'font.size': 12,
          'legend.fontsize': 12,
          'xtick.major.width': 1.5,
          'xtick.minor.width': 1,
          'xtick.major.size': 6,
          'xtick.minor.size': 3,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 1.5,
          'axes.linewidth': 1.5,
          'axes.labelweight': 1.5,
          'axes.titleweight': 1.5,
          'ytick.major.width': 1.5,
          'ytick.minor.width': 1,
          'ytick.major.size': 6,
          'ytick.minor.size': 3,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [8, 8],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

## 1. Get a Supernova from the truth table

Query the truth match catalog with the TAP service. Use coordinates near the center of DC2. Use a 0.5 degree radius. Use is_variable = 1 (true) to only return variables. Use is_variable = 1 (true) to only return variables. Use truth_type = 3 to only return Type Ia Supernovae. Use is_unique_truth_entry = 'true' to ensure good truth-table matches only. Use redshift < 0.3 to be more likely to get a full light curve with lots of data points.

Get the coordinates and all the useful information through the query. 

The schema browser for each table is here: https://dm.lsst.org/sdm_schemas/browser/dp02.html

**N.B.** New truth tables have been added to DP0.2. You may want to take a look at *delegate-contributions-dp02/desc_truth*

In [ ]:
%%time
results = service.search("SELECT ts.ra, ts.dec, ts.id, ts.host_galaxy, ts.redshift "\
                         "FROM dp02_dc2_catalogs.TruthSummary AS ts "
                         "WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec), "\
                         "CIRCLE('ICRS', 64.0, -35.5, 0.5)) = 1 "\
                         "AND ts.is_variable = 1 AND ts.truth_type = 3 AND ts.redshift < 0.3 ",\
                         maxrec=10000)

#### Show the TAP query results

In [ ]:
data = results.to_table().to_pandas()
data

### 1.1 Select a Supernova
Select the SN you want to analyze

In [ ]:
SN = data.loc[data['id'] == 'MS_9686_140'] # Very good object!


#SN = data.loc[data['id'] == 'MS_9686_138'] # Offset in g-band
#SN = data.loc[data['id'] == 'MS_9686_41'] # Offset in g-band and r-band
#SN = data.loc[data['id'] == 'MS_9686_103'] # Offset in g-band

### 1.2 Get useful information

In [ ]:
# Get the coordinates
SN_ra, SN_dec = SN['ra'].values[0]*u.deg, SN['dec'].values[0]*u.deg
targ_coord = SkyCoord(ra=SN_ra,dec=SN_dec)

### 1.3 Define the HTM ID for the Butler
Use these coordinates to define the HTM ID spatial search region to pass to the Butler's `queryDatasets` function.

In [ ]:
pixelization = sphgeom.HtmPixelization(15)
htm_id = pixelization.index(sphgeom.UnitVector3d(sphgeom.LonLat.fromDegrees(
                            targ_coord.ra.value, targ_coord.dec.value)))

# Obtain and print the scale to provide a sense of the size of the
#   sky pixelization being used
scale = pixelization.triangle(htm_id).getBoundingCircle().getOpeningAngle().asDegrees() * 3600
print(f'HTM ID={htm_id} at level={pixelization.getLevel()} is a ~{scale:0.2}" triangle.')

## 2. TAP Query the *DIA* tables and get the lightcurve
Every time a source is detected on a difference image with *Signal-to-noise ratio > 5*, a new entry is added to the `DiaSource` table.

Multiple detections associated to the same object will have the same *diaObjectId*, otherwise a new Id is created. 


The `DiaObject` table contains a single entry for each variable/transient object. 

Query the `DiaObject` table to get the Id of the Supernova (assuming a simple coordinate matching for simplicity). Then, use the retrieved Id to get the lightcurve from the `DiaSource` table.

### 2.1 Find the chosen Supernova on the *DiaObject* table

We only need the *DiaObjectId* for this query, and we will take the closest match within 0.5 arcsec.

In [ ]:
results = service.search("SELECT ra, decl, diaObjectId "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE CONTAINS(POINT('ICRS', ra, decl), "
                         "CIRCLE('ICRS'," + str(targ_coord.ra.deg) + ", "
                         + str(targ_coord.dec.deg) + ", 0.00014)) = 1 ", maxrec=100000)

DiaObjs = results.to_table()
del results

In [ ]:
DiaObjs

Get the object ID

In [ ]:
sel_objid = DiaObjs[0]['diaObjectId']
print(sel_objid)

In [ ]:
# ------ BONUS CELL --------

# Un-comment this if you want to see a light curve with template contamination,
# ideal to test Difference Imaging Task with other templates

sel_objid = 1650393341570843235

# Get the coordinates
SN_ra, SN_dec = 59.3569783*u.deg, -36.8152925*u.deg
targ_coord = SkyCoord(ra=SN_ra,dec=SN_dec)

pixelization = sphgeom.HtmPixelization(15)
htm_id = pixelization.index(sphgeom.UnitVector3d(sphgeom.LonLat.fromDegrees(
                            targ_coord.ra.value, targ_coord.dec.value)))

# Obtain and print the scale to provide a sense of the size of the
#   sky pixelization being used
scale = pixelization.triangle(htm_id).getBoundingCircle().getOpeningAngle().asDegrees() * 3600
print(f'HTM ID={htm_id} at level={pixelization.getLevel()} is a ~{scale:0.2}" triangle.')

### 2.2 Extract the Photometry from the DiaSource Table
To get a lightcurve, we query for times (midPointTai) and fluxes (with related errors). Rubin will provide fluxes in nanoJansky by default, but it is possible to convert them into magnitudes easily. 

There are two different types of measurement on the `DiaSource` table: *psFlux* is the PSF flux measured on the difference image, while *totFlux* is the PSF flux measured on the direct image (the calexp).


In [ ]:
results = service.search("SELECT ra, decl, diaObjectId, diaSourceId, ccdVisitId, "
                         "filterName, midPointTai, psFlux, psFluxErr, totFlux, totFluxErr, "
                         "scisql_nanojanskyToAbMag(psFlux) AS psAbMag, "
                         "scisql_nanojanskyToAbMag(totFlux) AS totAbMag, "
                         "scisql_nanojanskyToAbMagSigma(psFlux, psFluxErr) AS psAbMagErr, "
                         "scisql_nanojanskyToAbMagSigma(totFlux, totFluxErr) AS totAbMagErr "
                         "FROM dp02_dc2_catalogs.DiaSource "
                         "WHERE diaObjectId = "+str(sel_objid))

DiaSrcs = results.to_table()
del results

In [ ]:
# Get masks for different filters
u_band = (DiaSrcs['filterName'] == 'u')
g_band = (DiaSrcs['filterName'] == 'g')
r_band = (DiaSrcs['filterName'] == 'r')
i_band = (DiaSrcs['filterName'] == 'i')
z_band = (DiaSrcs['filterName'] == 'z')
y_band = (DiaSrcs['filterName'] == 'y')

In [ ]:
# Hot wo use list indexing to get values
print(DiaSrcs[g_band]['psFlux'].value)

### 2.3 Plot the lightcurves

Here we plot the lightcurves using both *psFlux* and *totFlux*. For a Supernova we do not expect a significant difference, but a comparison could be useful for many science cases.

In [ ]:
col = ['darkviolet','darkgreen','darkorange','red','brown','black']
bands = [u_band, g_band, r_band, i_band, z_band, y_band]

In [ ]:
fig = plt.figure(figsize=(8, 5))

for f,fil in enumerate( ['u','g','r','i','z','y'] ):
    
    # Simple scatter plot with points
    plt.scatter(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['psFlux'],
               s = 20, color = col[f], label = fil)
    # Light curve with error bars
    plt.errorbar(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['psFlux'],
                xerr = None, yerr = DiaSrcs[bands[f]]['psFluxErr'], lw = 0, 
                elinewidth = 1, marker = 'o', markersize = 4, capsize = 2, color = col[f])
    
plt.minorticks_on()
plt.xlabel('MJD (days)')
plt.ylabel('Flux_diff (nJy)')
plt.legend(loc='upper right')
plt.title("Multi-band lightcurve")
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))

for f,fil in enumerate( ['u','g','r','i','z','y'] ):
    
    # Simple scatter plot with points
    plt.scatter(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['totFlux'],
               s = 20, color = col[f], label = fil)
    # Light curve with error bars
    plt.errorbar(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['totFlux'],
                xerr = None, yerr = DiaSrcs[bands[f]]['totFluxErr'], lw = 0, 
                elinewidth = 1, marker = 'o', markersize = 4, capsize = 2, color = col[f])
    
plt.minorticks_on()
plt.xlabel('MJD (days)')
plt.ylabel('Flux_tot (nJy)')
plt.legend(loc='upper right')
plt.title("Multi-band lightcurve")
plt.show()

#### These are the same plots using magnitudes instead of fluxes

In [ ]:
fig = plt.figure(figsize=(8, 5))

for f,fil in enumerate( ['u','g','r','i','z','y'] ):
    
    # Simple scatter plot with points
    plt.scatter(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['psAbMag'],
               s = 20, color = col[f], label = fil)
    # Light curve with error bars
    plt.errorbar(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['psAbMag'],
                xerr = None, yerr = DiaSrcs[bands[f]]['psAbMagErr'], lw = 0, 
                elinewidth = 1, marker = 'o', markersize = 4, capsize = 2, color = col[f])

plt.minorticks_on()
plt.gca().invert_yaxis()
plt.xlabel('MJD (days)')
plt.ylabel('Mag_diff')
plt.legend(loc='upper right')
plt.title("Multi-band lightcurve")
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))

for f,fil in enumerate( ['u','g','r','i','z','y'] ):
    if f in [1,2,3]:
        # Simple scatter plot with points
        plt.scatter(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['totAbMag'],
                   s = 20, color = col[f], label = fil)
        # Light curve with error bars
        plt.errorbar(DiaSrcs[bands[f]]['midPointTai'], DiaSrcs[bands[f]]['totAbMag'],
                    xerr = None, yerr = DiaSrcs[bands[f]]['totAbMagErr'], lw = 0, 
                    elinewidth = 1, marker = 'o', markersize = 4, capsize = 2, color = col[f])
    else:
        continue
    
plt.minorticks_on()
plt.gca().invert_yaxis()
plt.xlabel('MJD (days)')
plt.ylabel('Mag_tot')
plt.legend(loc='upper right')
plt.title("Multi-band lightcurve")
plt.show()

### 2.4 Extract the Photometry from the ForcedSourceOnDiaObject Table
It may be useful to look at the lightcurve using Forced-photometry measurements on individual single-epoch visit images and difference images (whether or not the source has been detected above 5 sigma S/N). This information is provided in the `ForcedSourceOnDiaObject` table.

**N.B.** The table does not contain the time each exposure was observed. To build the lightcurve, we use a JOIN to extract visit info for each entry from the `CcdVisit` table.

In [ ]:
results = service.search("SELECT frc.coord_ra, frc.coord_dec, frc.diaObjectId, frc.ccdVisitId, "
                         "frc.psfDiffFlux, frc.psfDiffFluxErr, frc.psfFlux, frc.psfFluxErr, "
                         "scisql_nanojanskyToAbMag(psfDiffFlux) AS psAbMag, "
                         "scisql_nanojanskyToAbMag(psfFlux) AS totAbMag, "
                         "scisql_nanojanskyToAbMagSigma(psfDiffFlux, psfDiffFluxErr) AS psAbMagErr, "
                         "scisql_nanojanskyToAbMagSigma(psfFlux, psfFluxErr) AS totAbMagErr, "
                         "visinfo.ccdVisitId, visinfo.band, visinfo.expMidptMJD, visinfo.zeroPoint "
                         "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject AS frc "
                         "JOIN dp02_dc2_catalogs.CcdVisit as visinfo "
                         "ON visinfo.ccdVisitId = frc.ccdVisitId "
                         "WHERE frc.diaObjectId = " + str(sel_objid) +
                         "AND frc.psfDiffFlux_flag = " + str(0) +
                         "AND frc.psfFlux_flag = " + str(0))

ForcedSrcs = results.to_table()
del results

Let's plot the lightcurves as above

**N.B.** We omit error bars to make the plots easier to read 

In [ ]:
# Get masks for different filters
u_band_f = (ForcedSrcs['band'] == 'u')
g_band_f = (ForcedSrcs['band'] == 'g')
r_band_f = (ForcedSrcs['band'] == 'r')
i_band_f = (ForcedSrcs['band'] == 'i')
z_band_f = (ForcedSrcs['band'] == 'z')
y_band_f = (ForcedSrcs['band'] == 'y')

In [ ]:
col = ['darkviolet','darkgreen','darkorange','red','brown','black']
bands_f = [u_band_f, g_band_f, r_band_f, i_band_f, z_band_f, y_band_f]

In [ ]:
fig = plt.figure(figsize=(8, 5))

for f,fil in enumerate( ['u','g','r','i','z','y'] ):
    plt.plot(ForcedSrcs[bands_f[f]]['expMidptMJD'], ForcedSrcs[bands_f[f]]['psfDiffFlux'],
             marker = 'o', lw = 0, color = col[f], label = fil)
plt.minorticks_on()
plt.xlabel('MJD (days)')
plt.ylabel('Flux_diff (nJy)')
plt.legend(loc='upper right')
plt.title("Forced photometry lightcurve")

# Zoom around the peak
peak = np.argmax(ForcedSrcs[i_band_f]['psfDiffFlux'])
plt.xlim(ForcedSrcs[peak]['expMidptMJD']-100,ForcedSrcs[peak]['expMidptMJD']+100)
plt.ylim(-200)

plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))

for f,fil in enumerate( ['u','g','r','i','z','y'] ):
    plt.plot(ForcedSrcs[bands_f[f]]['expMidptMJD'], ForcedSrcs[bands_f[f]]['psAbMag'],
             marker = 'o', lw = 0, color = col[f], label = fil)
plt.minorticks_on()
plt.gca().invert_yaxis()
plt.xlabel('MJD (days)')
plt.ylabel('Mag_diff')
plt.legend(loc='upper right')
plt.title("Forced photometry lightcurve")

# Zoom around the peak (rembember the magnitude scale is reversed!)
peak = np.argmin(ForcedSrcs[i_band_f]['psAbMag'])
plt.xlim(ForcedSrcs[peak]['expMidptMJD']-100,ForcedSrcs[peak]['expMidptMJD']+100)
plt.ylim(30)

plt.show()

## 3. Get the images

We now show how to query the Butler to get images of the object of interest. We will start by looking at a deep coadd, then we will zoom in and query the Butler for the template, calexp and diffexp images for a selected point in the lightcurve.

### 3.1 Instantiate the Butler

In [ ]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collection)

### 3.2 Get the coadded image

In [ ]:
# Get tract and pacth (to retrieve the image)
results = service.search("SELECT patch, tract "
                         "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject "
                         "WHERE diaObjectId = "+str(sel_objid))

patch, tract = results.to_table()[0]
del results
print(patch, tract)

In [ ]:
coadd = butler.get('deepCoadd_ref',tract=tract,patch=patch)

In [ ]:
# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
# Select the filter and use tract and pacth from section 1.
dataId = {'tract': tract, 'patch': patch, 'band': 'i'}

# Retrieve the data using the `butler` instance and its function `get()`
coadd_calexp = butler.get('deepCoadd_calexp', **dataId)

In [ ]:
# Create a matplotlib.pyplot figure
fig, ax = plt.subplots()
# Define a colormap
afwDisplay.Display.setDefaultImageColormap('binary')
# Get an alias to the lsst.afw.display.Display() method
display = afwDisplay.Display(frame=fig)
# Set the image stretch algorithm and range
display.scale('asinh', 'zscale')
# Load the image into the display
display.mtv(coadd_calexp.image)
#plt.subplot(projection=WCS(coadd_calexp.getWcs().getFitsMetadata()))
plt.show()
# Clean up memory
remove_figure(fig)

In [ ]:
# Get pixel coordinates of the Supernova
RaDec = geom.SpherePoint(targ_coord.ra.degree,targ_coord.dec.degree,geom.degrees)

In [ ]:
# First, we need to extract the WCS solution, which provides the mapping
#   between XY pixel values and sky coordinates:
wcs = coadd_calexp.getWcs()

# Print the WCS info to see what it contains:
print(wcs)

In [ ]:
sn_xy = geom.Point2I(wcs.skyToPixel(RaDec))

### 3.3 Get a cutout around the coordinates of the Supernova

In [ ]:
# We will use the function definted in Section 0.
ra, dec = targ_coord.ra.degree, targ_coord.dec.degree
cutout_image = cutout_coadd(butler, ra, dec, datasetType='deepCoadd',
                            cutoutSideLength=300)
print("The size of the cutout in pixels is: ", cutout_image.image.array.shape)

In [ ]:
fig, ax = plt.subplots()
afwDisplay.Display.setDefaultImageColormap('binary')
afw_display = afwDisplay.Display(frame=fig)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(cutout_image.image)
#ax.axis('off')
plt.plot(sn_xy[0],sn_xy[1],'*',ms=10,color='cyan',label='SN')
plt.show()
remove_figure(fig)

### 3.4 Get template, calexp and diffexp for each point in the light curve

**N.B.** A new tutorial notebook by the Community Engagement Team on the image cutout service is in the works, and should avoid having to use the Butler for image access. Stay tooned for updates!

In [ ]:
# Get all the image identifiers for a selected filters
print(DiaSrcs[i_band]['ccdVisitId'].value)

In [ ]:
# Select a "visit" and a "ccd" id
sel_visit = int(str(DiaSrcs[i_band]['ccdVisitId'][0])[:-3])
sel_detector = int(str(DiaSrcs[i_band]['ccdVisitId'][0])[-3:])

In [ ]:
# Specify the type of image data product
datasetType = 'calexp'
# Specify the data that we are accessing
dataId = {'visit': sel_visit, 'detector': sel_detector}
# Retrieve the data using the `butler` instance and its function `get()`
calexp = butler.get(datasetType, dataId=dataId)

In [ ]:
# Specify the type of image data product
datasetType = 'goodSeeingDiff_differenceExp'
# Specify the data that we are accessing
dataId = {'visit': sel_visit, 'detector': sel_detector}
# Retrieve the data using the `butler` instance and its function `get()`
diffexp = butler.get(datasetType, dataId=dataId)

In [ ]:
# Specify the type of image data product
datasetType = 'goodSeeingDiff_templateExp'
# Specify the data that we are accessing
dataId = {'visit': sel_visit, 'detector': sel_detector}
# Retrieve the data using the `butler` instance and its function `get()`
template = butler.get(datasetType, dataId=dataId)

In [ ]:
# Specify the type of image data product
datasetType = 'sourceTable'
# Specify the data that we are accessing
dataId = {'visit': sel_visit, 'detector': sel_detector}
# Retrieve the data using the `butler` instance and its function `get()`
src = butler.get(datasetType, dataId=dataId)

In [ ]:
# Get pixel coordinates on the image
wcs2 = calexp.getWcs()
sn_xy_2 = geom.Point2I(wcs2.skyToPixel(RaDec))

### 3.5 Plot all the cutouts

In [ ]:
plt.rcParams.update({'font.size':20})
fig = plt.figure(figsize=(12, 7))
fig, ax = plt.subplots(1, 3, figsize=(24, 21))
afwDisplay.Display.setDefaultImageColormap('gray')

plt.sca(ax[0])  # Next sub-plot
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(template.image)
plt.title('template')
# Centre the image around the transient and limit borders
plt.xlim(sn_xy_2[0]-100, sn_xy_2[0]+100)
plt.ylim(sn_xy_2[1]-100, sn_xy_2[1]+100)

plt.sca(ax[1])  # Next sub-plot
display2 = afwDisplay.Display(frame=fig)
display2.scale('linear', 'zscale')
display2.mtv(calexp.image)
plt.title('exposure')
# Centre the image around the transient and limit borders
plt.xlim(sn_xy_2[0]-100, sn_xy_2[0]+100)
plt.ylim(sn_xy_2[1]-100, sn_xy_2[1]+100)

plt.sca(ax[2])  # Next sub-plot
display3 = afwDisplay.Display(frame=fig)
display3.scale('linear', 'zscale')
display3.mtv(diffexp.image)
# Centre the image around the transient and limit borders
plt.xlim(sn_xy_2[0]-100, sn_xy_2[0]+100)
plt.ylim(sn_xy_2[1]-100, sn_xy_2[1]+100)
plt.title('difference')

plt.tight_layout()
plt.show()
# Clean up memory
remove_figure(fig)

## 4. Launch the *Difference Imaging Task* using your own templates

It is possible to produce difference images on the RSP by selecting a coadd (or another calexp) as templates.

This is a DRAFT as it uses the deprecated classe *ImageDifferenceTask*. Updates of this notebook may results from work within the TVS DP0 Task Force

Package info: https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/imageDifference.py

In [ ]:
# Select a "visit" and a "ccd" id for a new_calexp to be used as a template
sel_visit = int(str(DiaSrcs[i_band]['ccdVisitId'][4])[:-3])
sel_detector = int(str(DiaSrcs[i_band]['ccdVisitId'][4])[-3:])

dataId = {'visit': sel_visit, 'detector': sel_detector}
calexp_new = butler.get('calexp', dataId=dataId)

src_new = butler.get('sourceTable', dataId=dataId)

In [ ]:
# Get pixel coordinates on the image
wcs2 = calexp_new.getWcs()
sn_xy_2 = geom.Point2I(wcs2.skyToPixel(RaDec))

In [ ]:
from lsst.pipe.tasks.imageDifference import ImageDifferenceConfig, ImageDifferenceTask

imdiff_config = ImageDifferenceConfig()
imdiff_task = ImageDifferenceTask(imdiff_config)

#res = imdiff_task.run(exposure=calexp, templateExposure=template)

#res = imdiff_task.run(exposure=calexp_new, templateExposure=coadd_calexp) # Working, but the template is contaminated as well
res = imdiff_task.run(exposure=calexp_new, templateExposure=calexp) # Working using another calexp as a template

In [ ]:
plt.rcParams.update({'font.size':20})
fig = plt.figure(figsize=(12, 7))
fig, ax = plt.subplots(1, 3, figsize=(24, 21))
afwDisplay.Display.setDefaultImageColormap('gray')

plt.sca(ax[0])  # Next sub-plot
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
#display1.mtv(calexp_orig_i.image)
display1.mtv(res.warpedExposure.image)
plt.title('template)')
# Centre the image around the transient and limit borders
plt.xlim(sn_xy_2[0]-100, sn_xy_2[0]+100)
plt.ylim(sn_xy_2[1]-100, sn_xy_2[1]+100)

plt.sca(ax[1])  # Next sub-plot
display2 = afwDisplay.Display(frame=fig)
display2.scale('linear', 'zscale')
display2.mtv(calexp_new.image)
plt.title('exposure')
# Centre the image around the transient and limit borders
plt.xlim(sn_xy_2[0]-100, sn_xy_2[0]+100)
plt.ylim(sn_xy_2[1]-100, sn_xy_2[1]+100)

plt.sca(ax[2])  # Next sub-plot
display3 = afwDisplay.Display(frame=fig)
display3.scale('linear', 'zscale')
display3.mtv(res.subtractedExposure.image)
# Centre the image around the transient and limit borders
plt.xlim(sn_xy_2[0]-100, sn_xy_2[0]+100)
plt.ylim(sn_xy_2[1]-100, sn_xy_2[1]+100)
plt.title('difference')

plt.tight_layout()
plt.show()

### 4.1 Test with the new DIA package (DRAFT)

This new task returns unexpected errors.

Possible work in progress for the TVS DP0 Task Force to understand how to use it and produce new tutorial notebooks.

Package info: https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/subtractImages.py

In [ ]:
from lsst.ip.diffim import subtractImages, detectAndMeasure, zogy

In [ ]:
imdiff_config = subtractImages.AlardLuptonSubtractConfig()

In [ ]:
imdiff_task = subtractImages.AlardLuptonSubtractTask()

In [ ]:
dia = imdiff_task.run(template=template, science=calexp, sources=src)

In [ ]:
dia = imdiff_task.run(template=calexp, science=calexp_new, sources=src_new)